# 🚀 MLOps und Model Deployment
### Woche 7, Notebook 1: Von der Entwicklung zur Produktion

**Lernziele:**
- 🔄 **MLOps-Pipeline**: Verstehen moderner ML-Deployment-Praktiken
- 🐳 **Docker & Containerisierung**: Reproduzierbare ML-Umgebungen erstellen
- 🌐 **API-Deployment**: ML-Modelle über REST APIs bereitstellen
- 📊 **Model Monitoring**: Performance-Überwachung in Produktion
- ⚡ **CI/CD für ML**: Automatisierte Deployment-Pipelines

---

**Was Sie nach diesem Notebook können:**
- ML-Modelle für Produktionsumgebungen vorbereiten
- Docker-Container für ML-Services erstellen
- REST APIs für Model Serving implementieren
- Model Performance überwachen und loggen
- Deployment-Strategien für verschiedene Umgebungen anwenden

---

> 💡 **Praxis-Tipp**: In diesem Notebook erstellen wir ein vollständiges Deployment-Setup für Ihr Abschlussprojekt!

## 🔄 Was ist MLOps?

**MLOps** (Machine Learning Operations) kombiniert Machine Learning, DevOps und Data Engineering, um ML-Systeme zuverlässig und effizient in Produktion zu bringen.

### 🎯 Kernprinzipien von MLOps

1. **🔁 Automatisierung**: Von Training bis Deployment
2. **🔍 Versionierung**: Code, Daten und Modelle
3. **📊 Monitoring**: Performance und Data Drift
4. **🧪 Testing**: Validierung und A/B-Tests
5. **🚀 Continuous Deployment**: Schnelle, sichere Releases

### 📈 MLOps vs. Traditional DevOps

| Aspekt | Traditional DevOps | MLOps |
|--------|-------------------|-------|
| **Artefakte** | Code, Configs | Code + Daten + Modelle |
| **Testing** | Unit/Integration Tests | Data/Model Validation |
| **Deployment** | Statische Anwendungen | Dynamische ML-Services |
| **Monitoring** | System Metrics | Model Performance + Drift |

---

### 🛠️ MLOps Toolchain (2025)

```mermaid
graph LR
    A[Data] --> B[Preprocessing]
    B --> C[Training]
    C --> D[Validation]
    D --> E[Containerization]
    E --> F[Deployment]
    F --> G[Monitoring]
    G --> A
```

**Beispiel-Tools:**
- **Experiment Tracking**: MLflow, Weights & Biases
- **Model Registry**: MLflow, DVC
- **Containerization**: Docker, Kubernetes
- **Deployment**: FastAPI, Flask, Streamlit
- **Monitoring**: Prometheus, Grafana, Evidently

In [ ]:
# 📦 Setup und Imports für MLOps
import os
import pickle
import json
import joblib
from datetime import datetime
import logging

# ML und Data Science
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_iris

# API und Web Framework
try:
    from fastapi import FastAPI, HTTPException
    from pydantic import BaseModel
    import uvicorn
    print("✅ FastAPI verfügbar")
except ImportError:
    print("⚠️ FastAPI nicht installiert - kann über pip install fastapi uvicorn installiert werden")

# Monitoring und Logging
try:
    import mlflow
    import mlflow.sklearn
    print("✅ MLflow verfügbar")
except ImportError:
    print("⚠️ MLflow nicht installiert - kann über pip install mlflow installiert werden")

# Utilities
import warnings
warnings.filterwarnings('ignore')

print("✅ Basis MLOps-Bibliotheken erfolgreich importiert!")
print(f"📅 Setup-Zeit: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Logging Setup für Produktionsumgebung
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

## 🔄 MLOps-Lifecycle in der Praxis

### 1️⃣ Model Development Pipeline

Wir implementieren eine vollständige MLOps-Pipeline mit einem **Iris Classification Modell** als Beispiel.

In [ ]:
# 🔬 Model Development mit Tracking

class MLOpsModelPipeline:
    """
    Vollständige MLOps-Pipeline für Model Development und Deployment
    """
    
    def __init__(self, experiment_name="iris_classification"):
        self.experiment_name = experiment_name
        self.model = None
        self.model_version = None
        self.model_path = None
        
    def load_and_prepare_data(self):
        """Daten laden und vorbereiten"""
        logger.info("📊 Lade und bereite Daten vor...")
        
        # Iris Dataset laden
        iris = load_iris()
        X, y = iris.data, iris.target
        
        # Train/Test Split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test
        self.feature_names = iris.feature_names
        self.target_names = iris.target_names
        
        logger.info(f"✅ Daten geladen: {len(X_train)} Training, {len(X_test)} Test Samples")
        
        return X_train, X_test, y_train, y_test
    
    def train_model(self, n_estimators=100):
        """Model Training mit Tracking"""
        logger.info("🏋️ Starte Model Training...")
        
        # Model Training
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            random_state=42
        )
        model.fit(self.X_train, self.y_train)
        
        # Predictions
        y_pred = model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        
        # Model speichern
        self.model = model
        self.model_path = f"models/iris_classifier_{datetime.now().strftime('%Y%m%d_%H%M%S')}.joblib"
        os.makedirs('models', exist_ok=True)
        joblib.dump(model, self.model_path)
        
        # Logging (falls MLflow verfügbar)
        training_info = {
            "n_estimators": n_estimators,
            "algorithm": "RandomForest",
            "accuracy": accuracy,
            "train_size": len(self.X_train),
            "test_size": len(self.X_test),
            "model_path": self.model_path
        }
        
        logger.info(f"✅ Model trainiert! Accuracy: {accuracy:.4f}")
        logger.info(f"💾 Model gespeichert: {self.model_path}")
        
        return model, accuracy, training_info
    
    def evaluate_model(self):
        """Detaillierte Model Evaluation"""
        if self.model is None:
            raise ValueError("Model muss zuerst trainiert werden!")
        
        y_pred = self.model.predict(self.X_test)
        
        # Classification Report
        report = classification_report(
            self.y_test, y_pred, 
            target_names=self.target_names,
            output_dict=True
        )
        
        print("📊 Model Evaluation Report:")
        print("=" * 50)
        print(classification_report(self.y_test, y_pred, target_names=self.target_names))
        
        return report

# Pipeline initialisieren und ausführen
pipeline = MLOpsModelPipeline()
X_train, X_test, y_train, y_test = pipeline.load_and_prepare_data()
model, accuracy, training_info = pipeline.train_model()

print(f"🎯 Model Accuracy: {accuracy:.4f}")
print(f"🔢 Feature Names: {pipeline.feature_names}")
print(f"🏷️  Target Classes: {pipeline.target_names}")

# Detaillierte Evaluation
evaluation_report = pipeline.evaluate_model()

### 2️⃣ Model Validation und Testing

Bevor wir ein Modell in Produktion bringen, müssen wir es gründlich validieren.

In [ ]:
# 🧪 Model Validation und Testing

class ModelValidator:
    """
    Klasse für umfassende Model Validation
    """
    
    def __init__(self, model, X_test, y_test, feature_names, target_names):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.feature_names = feature_names
        self.target_names = target_names
    
    def validate_input_schema(self, X_input):
        """Validiert das Input-Schema"""
        expected_features = len(self.feature_names)
        
        if X_input.shape[1] != expected_features:
            raise ValueError(
                f"Input hat {X_input.shape[1]} Features, "
                f"erwartet werden {expected_features}"
            )
        
        logger.info("✅ Input Schema Validation erfolgreich")
        return True
    
    def validate_prediction_range(self, predictions):
        """Validiert den Vorhersagebereich"""
        valid_classes = set(range(len(self.target_names)))
        pred_classes = set(predictions)
        
        if not pred_classes.issubset(valid_classes):
            invalid_classes = pred_classes - valid_classes
            raise ValueError(f"Ungültige Klassen vorhergesagt: {invalid_classes}")
        
        logger.info("✅ Prediction Range Validation erfolgreich")
        return True
    
    def performance_regression_test(self, min_accuracy=0.85):
        """Performance Regression Test"""
        predictions = self.model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        
        if accuracy < min_accuracy:
            raise ValueError(
                f"Model Performance zu niedrig: {accuracy:.4f} < {min_accuracy}"
            )
        
        logger.info(f"✅ Performance Test bestanden: {accuracy:.4f} >= {min_accuracy}")
        return accuracy
    
    def run_all_validations(self, min_accuracy=0.85):
        """Führt alle Validierungen durch"""
        print("🧪 Starte umfassende Model Validation...")
        print("=" * 50)
        
        try:
            # Schema Validation
            self.validate_input_schema(self.X_test)
            
            # Predictions generieren
            predictions = self.model.predict(self.X_test)
            
            # Range Validation
            self.validate_prediction_range(predictions)
            
            # Performance Test
            accuracy = self.performance_regression_test(min_accuracy)
            
            print(f"🎉 Alle Validierungen erfolgreich! Model ist produktionsbereit.")
            print(f"🎯 Finale Accuracy: {accuracy:.4f}")
            
            return True, accuracy
            
        except Exception as e:
            print(f"❌ Validation fehlgeschlagen: {str(e)}")
            return False, None

# Model Validation durchführen
validator = ModelValidator(
    model=pipeline.model,
    X_test=pipeline.X_test,
    y_test=pipeline.y_test,
    feature_names=pipeline.feature_names,
    target_names=pipeline.target_names
)

validation_passed, final_accuracy = validator.run_all_validations(min_accuracy=0.85)

### 3️⃣ Model Serving mit FastAPI

Jetzt erstellen wir eine REST API für unser Modell.

In [ ]:
# 🌐 Model Serving mit FastAPI

# Datenmodelle für API
from pydantic import BaseModel
from typing import List

class IrisFeatures(BaseModel):
    """Eingabe-Schema für Iris Prediction"""
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float
    
    class Config:
        schema_extra = {
            "example": {
                "sepal_length": 5.1,
                "sepal_width": 3.5,
                "petal_length": 1.4,
                "petal_width": 0.2
            }
        }

class PredictionResponse(BaseModel):
    """Ausgabe-Schema für Prediction"""
    prediction: int
    prediction_label: str
    confidence: float
    model_version: str
    timestamp: str

# Model Serving Klasse
class IrisModelServer:
    """
    Production-ready Model Server
    """
    
    def __init__(self, model_path: str, target_names: List[str]):
        self.model = joblib.load(model_path)
        self.target_names = target_names
        self.model_version = os.path.basename(model_path)
        logger.info(f"✅ Model geladen: {model_path}")
    
    def predict(self, features: IrisFeatures) -> PredictionResponse:
        """Einzelne Prediction"""
        try:
            # Features zu Array konvertieren
            X = np.array([[
                features.sepal_length,
                features.sepal_width,
                features.petal_length,
                features.petal_width
            ]])
            
            # Prediction
            prediction = self.model.predict(X)[0]
            probabilities = self.model.predict_proba(X)[0]
            confidence = float(np.max(probabilities))
            
            # Response erstellen
            response = PredictionResponse(
                prediction=int(prediction),
                prediction_label=self.target_names[prediction],
                confidence=confidence,
                model_version=self.model_version,
                timestamp=datetime.now().isoformat()
            )
            
            logger.info(f"Prediction: {prediction} ({self.target_names[prediction]}) - Confidence: {confidence:.4f}")
            return response
            
        except Exception as e:
            logger.error(f"Prediction Fehler: {str(e)}")
            raise HTTPException(status_code=500, detail=f"Prediction fehlgeschlagen: {str(e)}")

# Model Server initialisieren
model_server = IrisModelServer(
    model_path=pipeline.model_path,
    target_names=pipeline.target_names.tolist()
)

# Test Prediction
test_features = IrisFeatures(
    sepal_length=5.1,
    sepal_width=3.5,
    petal_length=1.4,
    petal_width=0.2
)

test_prediction = model_server.predict(test_features)
print("🧪 Test Prediction:")
print(f"   Klasse: {test_prediction.prediction_label}")
print(f"   Confidence: {test_prediction.confidence:.4f}")
print(f"   Model Version: {test_prediction.model_version}")

### 4️⃣ FastAPI Application erstellen

Jetzt erstellen wir die vollständige FastAPI Anwendung.

In [ ]:
# 🚀 FastAPI Application für Model Serving

# FastAPI App erstellen
app = FastAPI(
    title="Iris Classification API",
    description="Production-ready ML API für Iris Klassifikation",
    version="1.0.0"
)

# Global model server
global_model_server = None

@app.on_event("startup")
async def startup_event():
    """Initialisierung beim App-Start"""
    global global_model_server
    global_model_server = model_server
    logger.info("🚀 Iris Classification API gestartet")

@app.get("/")
async def root():
    """Health Check Endpoint"""
    return {
        "message": "Iris Classification API",
        "status": "healthy",
        "model_version": global_model_server.model_version if global_model_server else None,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health_check():
    """Detaillierter Health Check"""
    if global_model_server is None:
        raise HTTPException(status_code=503, detail="Model Server nicht verfügbar")
    
    return {
        "status": "healthy",
        "model_loaded": True,
        "model_version": global_model_server.model_version,
        "target_classes": global_model_server.target_names,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict(features: IrisFeatures):
    """Iris Klassifikation Endpoint"""
    if global_model_server is None:
        raise HTTPException(status_code=503, detail="Model Server nicht verfügbar")
    
    return global_model_server.predict(features)

@app.post("/predict/batch")
async def predict_batch(features_list: List[IrisFeatures]):
    """Batch Prediction Endpoint"""
    if global_model_server is None:
        raise HTTPException(status_code=503, detail="Model Server nicht verfügbar")
    
    predictions = []
    for features in features_list:
        prediction = global_model_server.predict(features)
        predictions.append(prediction)
    
    return {
        "predictions": predictions,
        "batch_size": len(predictions),
        "timestamp": datetime.now().isoformat()
    }

print("✅ FastAPI Application erstellt!")
print("📝 Verfügbare Endpoints:")
print("   GET  /              - Root endpoint")
print("   GET  /health        - Health check")
print("   POST /predict       - Einzelne Prediction")
print("   POST /predict/batch - Batch Predictions")

# API Dokumentation anzeigen
print("\n📚 API Dokumentation verfügbar unter:")
print("   http://localhost:8000/docs (Swagger UI)")
print("   http://localhost:8000/redoc (ReDoc)")

### 5️⃣ Dockerization

Jetzt erstellen wir Docker-Container für reproduzierbare Deployments.

In [ ]:
# 🐳 Dockerfile und Docker Deployment

dockerfile_content = """
# Production Dockerfile für ML API
FROM python:3.11-slim

# System Dependencies
RUN apt-get update && apt-get install -y \
    gcc \
    && rm -rf /var/lib/apt/lists/*

# Working Directory
WORKDIR /app

# Requirements kopieren und installieren
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Anwendungscode kopieren
COPY . .

# Model Directory erstellen
RUN mkdir -p models

# Port exposition
EXPOSE 8000

# Health Check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:8000/health || exit 1

# Anwendung starten
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

# Docker-Compose für Development
docker_compose_content = """
version: '3.8'

services:
  iris-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - ENV=development
      - LOG_LEVEL=info
    volumes:
      - ./models:/app/models
      - ./logs:/app/logs
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3

  # Optional: Monitoring mit Prometheus
  prometheus:
    image: prom/prometheus:latest
    ports:
      - "9090:9090"
    volumes:
      - ./monitoring/prometheus.yml:/etc/prometheus/prometheus.yml
    command:
      - '--config.file=/etc/prometheus/prometheus.yml'
      - '--storage.tsdb.path=/prometheus'
      - '--web.console.libraries=/etc/prometheus/console_libraries'
      - '--web.console.templates=/etc/prometheus/consoles'

  # Optional: Grafana für Visualisierung
  grafana:
    image: grafana/grafana:latest
    ports:
      - "3000:3000"
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin
    volumes:
      - grafana-storage:/var/lib/grafana

volumes:
  grafana-storage:
"""

# Requirements für Production
requirements_content = """
fastapi==0.104.1
uvicorn[standard]==0.24.0
scikit-learn==1.3.2
pandas==2.1.4
numpy==1.24.3
joblib==1.3.2
pydantic==2.5.0
python-multipart==0.0.6
"""

# Dateien erstellen
os.makedirs('docker_deployment', exist_ok=True)

with open('docker_deployment/Dockerfile', 'w') as f:
    f.write(dockerfile_content)

with open('docker_deployment/docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)

with open('docker_deployment/requirements.txt', 'w') as f:
    f.write(requirements_content)

print("🐳 Docker-Dateien erstellt:")
print("   📄 docker_deployment/Dockerfile")
print("   📄 docker_deployment/docker-compose.yml")
print("   📄 docker_deployment/requirements.txt")

print("\n🚀 Deployment Commands:")
print("   docker build -t iris-api .")
print("   docker run -p 8000:8000 iris-api")
print("   docker-compose up -d")

### 6️⃣ Model Monitoring und Logging

Für Produktionsumgebungen ist Monitoring essentiell.

In [ ]:
# 📊 Model Monitoring und Performance Tracking

import time
from collections import defaultdict
import threading

class ModelMonitor:
    """
    Model Performance Monitor für Production
    """
    
    def __init__(self):
        self.prediction_count = 0
        self.prediction_times = []
        self.prediction_confidences = []
        self.class_distribution = defaultdict(int)
        self.errors = []
        self.lock = threading.Lock()
    
    def log_prediction(self, prediction_time: float, confidence: float, predicted_class: str):
        """Prediction Metriken loggen"""
        with self.lock:
            self.prediction_count += 1
            self.prediction_times.append(prediction_time)
            self.prediction_confidences.append(confidence)
            self.class_distribution[predicted_class] += 1
    
    def log_error(self, error_message: str):
        """Fehler loggen"""
        with self.lock:
            self.errors.append({
                'timestamp': datetime.now().isoformat(),
                'error': error_message
            })
    
    def get_metrics(self):
        """Aktuelle Metriken abrufen"""
        with self.lock:
            if not self.prediction_times:
                return {
                    'total_predictions': 0,
                    'avg_prediction_time': 0,
                    'avg_confidence': 0,
                    'class_distribution': {},
                    'error_count': len(self.errors)
                }
            
            return {
                'total_predictions': self.prediction_count,
                'avg_prediction_time': np.mean(self.prediction_times),
                'max_prediction_time': np.max(self.prediction_times),
                'min_prediction_time': np.min(self.prediction_times),
                'avg_confidence': np.mean(self.prediction_confidences),
                'min_confidence': np.min(self.prediction_confidences),
                'class_distribution': dict(self.class_distribution),
                'error_count': len(self.errors),
                'recent_errors': self.errors[-5:] if self.errors else []
            }

# Enhanced Model Server mit Monitoring
class MonitoredIrisModelServer(IrisModelServer):
    """
    Model Server mit integriertem Monitoring
    """
    
    def __init__(self, model_path: str, target_names: List[str]):
        super().__init__(model_path, target_names)
        self.monitor = ModelMonitor()
    
    def predict(self, features: IrisFeatures) -> PredictionResponse:
        """Prediction mit Monitoring"""
        start_time = time.time()
        
        try:
            response = super().predict(features)
            
            # Monitoring Daten loggen
            prediction_time = time.time() - start_time
            self.monitor.log_prediction(
                prediction_time=prediction_time,
                confidence=response.confidence,
                predicted_class=response.prediction_label
            )
            
            return response
            
        except Exception as e:
            self.monitor.log_error(str(e))
            raise
    
    def get_monitoring_data(self):
        """Monitoring Daten abrufen"""
        return self.monitor.get_metrics()

# Monitored Model Server erstellen
monitored_server = MonitoredIrisModelServer(
    model_path=pipeline.model_path,
    target_names=pipeline.target_names.tolist()
)

# Test mit mehreren Predictions
print("🧪 Testing mit Monitoring...")
test_cases = [
    IrisFeatures(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2),
    IrisFeatures(sepal_length=7.0, sepal_width=3.2, petal_length=4.7, petal_width=1.4),
    IrisFeatures(sepal_length=6.3, sepal_width=3.3, petal_length=6.0, petal_width=2.5)
]

for i, test_case in enumerate(test_cases):
    prediction = monitored_server.predict(test_case)
    print(f"   Test {i+1}: {prediction.prediction_label} (Confidence: {prediction.confidence:.4f})")

# Monitoring Metriken anzeigen
metrics = monitored_server.get_monitoring_data()
print("\n📊 Monitoring Metriken:")
print(f"   Total Predictions: {metrics['total_predictions']}")
print(f"   Avg Prediction Time: {metrics['avg_prediction_time']:.4f}s")
print(f"   Avg Confidence: {metrics['avg_confidence']:.4f}")
print(f"   Class Distribution: {metrics['class_distribution']}")
print(f"   Errors: {metrics['error_count']}")

### 7️⃣ Streamlit Dashboard für Monitoring

In [ ]:
# 📊 Streamlit Monitoring Dashboard erstellen

streamlit_dashboard = """
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
import json
from datetime import datetime
import time

st.set_page_config(
    page_title="MLOps Monitoring Dashboard",
    page_icon="📊",
    layout="wide"
)

st.title("🚀 MLOps Monitoring Dashboard")
st.markdown("**Real-time Monitoring für Iris Classification API**")

# Sidebar für Konfiguration
st.sidebar.header("⚙️ Konfiguration")
api_url = st.sidebar.text_input("API URL", "http://localhost:8000")
refresh_interval = st.sidebar.slider("Refresh Interval (s)", 5, 60, 10)

# Auto-refresh
if st.sidebar.button("🔄 Refresh"):
    st.rerun()

# API Health Check
try:
    health_response = requests.get(f"{api_url}/health", timeout=5)
    if health_response.status_code == 200:
        health_data = health_response.json()
        st.success(f"✅ API Status: {health_data['status']}")
        
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Model Version", health_data.get('model_version', 'N/A'))
        with col2:
            st.metric("Model Loaded", "✅" if health_data.get('model_loaded') else "❌")
        with col3:
            st.metric("Target Classes", len(health_data.get('target_classes', [])))
    else:
        st.error(f"❌ API nicht erreichbar (Status: {health_response.status_code})")
except Exception as e:
    st.error(f"❌ Verbindung zur API fehlgeschlagen: {str(e)}")

st.divider()

# Prediction Interface
st.header("🧪 Model Testing")

col1, col2 = st.columns(2)

with col1:
    st.subheader("Input Features")
    sepal_length = st.slider("Sepal Length", 4.0, 8.0, 5.1, 0.1)
    sepal_width = st.slider("Sepal Width", 2.0, 4.5, 3.5, 0.1)
    petal_length = st.slider("Petal Length", 1.0, 7.0, 1.4, 0.1)
    petal_width = st.slider("Petal Width", 0.1, 2.5, 0.2, 0.1)
    
    if st.button("🔮 Predict", type="primary"):
        try:
            prediction_data = {
                "sepal_length": sepal_length,
                "sepal_width": sepal_width,
                "petal_length": petal_length,
                "petal_width": petal_width
            }
            
            response = requests.post(
                f"{api_url}/predict",
                json=prediction_data,
                timeout=10
            )
            
            if response.status_code == 200:
                result = response.json()
                st.success(f"Prediction: **{result['prediction_label']}**")
                st.info(f"Confidence: {result['confidence']:.4f}")
                st.caption(f"Timestamp: {result['timestamp']}")
            else:
                st.error(f"Prediction fehlgeschlagen: {response.text}")
                
        except Exception as e:
            st.error(f"Fehler bei Prediction: {str(e)}")

with col2:
    st.subheader("Iris Species")
    st.image("https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg", 
             caption="Iris Versicolor", width=300)

st.divider()

# Performance Metriken (Simulation)
st.header("📈 Performance Metriken")

# Simulierte Daten für Demo
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', periods=30, freq='D')
predictions_per_day = np.random.poisson(100, 30)
avg_response_time = np.random.normal(0.05, 0.01, 30)
avg_confidence = np.random.normal(0.92, 0.05, 30)

col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        "Total Predictions (30d)",
        f"{predictions_per_day.sum():,}",
        delta=f"+{predictions_per_day[-1] - predictions_per_day[-2]}"
    )

with col2:
    st.metric(
        "Avg Response Time",
        f"{avg_response_time[-1]:.3f}s",
        delta=f"{avg_response_time[-1] - avg_response_time[-2]:+.3f}s"
    )

with col3:
    st.metric(
        "Avg Confidence",
        f"{avg_confidence[-1]:.3f}",
        delta=f"{avg_confidence[-1] - avg_confidence[-2]:+.3f}"
    )

with col4:
    error_rate = np.random.uniform(0.001, 0.005)
    st.metric(
        "Error Rate",
        f"{error_rate:.3%}",
        delta="-0.001%"
    )

# Charts
col1, col2 = st.columns(2)

with col1:
    # Predictions over time
    fig_predictions = px.line(
        x=dates, y=predictions_per_day,
        title="📊 Predictions pro Tag",
        labels={'x': 'Datum', 'y': 'Anzahl Predictions'}
    )
    st.plotly_chart(fig_predictions, use_container_width=True)

with col2:
    # Response time over time
    fig_response = px.line(
        x=dates, y=avg_response_time,
        title="⚡ Response Time Trend",
        labels={'x': 'Datum', 'y': 'Response Time (s)'}
    )
    st.plotly_chart(fig_response, use_container_width=True)

# Class Distribution
st.subheader("🎯 Prediction Class Distribution")
class_counts = {
    'setosa': np.random.poisson(200),
    'versicolor': np.random.poisson(180),
    'virginica': np.random.poisson(220)
}

fig_pie = px.pie(
    values=list(class_counts.values()),
    names=list(class_counts.keys()),
    title="Class Distribution (Last 30 Days)"
)
st.plotly_chart(fig_pie, use_container_width=True)

# Footer
st.divider()
st.caption(f"Last updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
"""

# Streamlit Dashboard Datei erstellen
with open('07_01_streamlit_mlops_dashboard.py', 'w') as f:
    f.write(streamlit_dashboard)

print("📊 Streamlit Monitoring Dashboard erstellt!")
print("📄 Datei: 07_01_streamlit_mlops_dashboard.py")
print("\n🚀 Dashboard starten mit:")
print("   streamlit run 07_01_streamlit_mlops_dashboard.py")
print("\n📱 Dashboard Features:")
print("   ✅ API Health Monitoring")
print("   🧪 Interactive Model Testing")
print("   📈 Performance Metriken")
print("   🎯 Prediction Analytics")
print("   🔄 Real-time Updates")

## 🎯 Portfolio-Zusammenfassung: MLOps & Deployment

### 🏆 Was Sie gelernt haben

1. **🔄 MLOps-Pipeline**: Vollständigen ML-Lifecycle von Training bis Deployment
2. **🐳 Containerisierung**: Docker-Setup für reproduzierbare ML-Services
3. **🌐 API-Development**: FastAPI für robuste ML-Endpoints
4. **📊 Model Monitoring**: Performance-Tracking in Produktionsumgebungen
5. **📱 Dashboard-Entwicklung**: Streamlit für ML-Monitoring

### 🛠️ Praktische Fähigkeiten

- ✅ **Model Validation** - Automatisierte Tests für ML-Modelle
- ✅ **API Design** - RESTful Services für ML-Predictions
- ✅ **Docker Deployment** - Containerisierte ML-Anwendungen
- ✅ **Monitoring Setup** - Performance- und Error-Tracking
- ✅ **Production Readiness** - Robuste, skalierbare ML-Services

### 📁 Generierte Artefakte

1. **🤖 Trainiertes Model** - `models/iris_classifier_*.joblib`
2. **🌐 FastAPI Application** - REST API für Model Serving
3. **🐳 Docker Setup** - `docker_deployment/` Ordner
4. **📊 Monitoring Dashboard** - `07_01_streamlit_mlops_dashboard.py`
5. **📝 Deployment Documentation** - Production-ready Setup

### 🚀 Nächste Schritte für Ihr Projekt

1. **🔧 Anpassung**: MLOps-Pipeline für Ihr eigenes Modell adaptieren
2. **☁️ Cloud Deployment**: AWS/Azure/GCP Integration
3. **📈 Advanced Monitoring**: Drift Detection, A/B Testing
4. **🔄 CI/CD Integration**: GitHub Actions für automatisches Deployment
5. **🔒 Security**: Authentication, Rate Limiting, Encryption

---

> 💡 **Portfolio-Tipp**: Dokumentieren Sie Ihre MLOps-Pipeline ausführlich - das zeigt Arbeitgebern Ihre Production-Ready Skills!

### 📚 Weiterführende Ressourcen

- **MLOps Best Practices**: [Google MLOps Guide](https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)
- **FastAPI Documentation**: [fastapi.tiangolo.com](https://fastapi.tiangolo.com/)
- **Docker Best Practices**: [Docker Documentation](https://docs.docker.com/develop/dev-best-practices/)
- **Model Monitoring**: [Evidently AI](https://evidentlyai.com/)
- **Kubernetes für ML**: [Kubeflow](https://www.kubeflow.org/)